In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
fake=pd.read_csv('/content/drive/My Drive/colab/Fake.csv')
true=pd.read_csv('/content/drive/My Drive/colab/True.csv')

In [ ]:
true['label'] = 0
fake['label'] = 1

In [ ]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [ ]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [ ]:
dataset1 = true[['text','label']]
dataset2 = fake[['text','label']]

In [ ]:
dataset = pd.concat([dataset1,dataset2])
dataset.shape

(44898, 2)

In [ ]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
dataset['label'].value_counts()

1    23481
0    21417
Name: label, dtype: int64

In [ ]:
dataset = dataset.sample(frac=1)

In [ ]:
dataset.head(20)

,text,label
14181,AUDIO OF KARL ROVE ON HUGH HEWITT Picking ...,1
20919,HYSTERICAL!A US Marine enters the Catholic Chu...,1
1655,Alex Jones of the conspiracy theory website In...,1
14675,"This isn t just any church organization, this ...",1
14862,The green criminals who re working on how to d...,1
15060,BEIJING (Reuters) - President Xi Jinping vowed...,0
8327,"Yesterday, Republican presidential front runne...",1
6802,Fox host Sean Hannity lost his cool during a r...,1
5806,NEW YORK (Reuters) - Supreme Court nominee Jud...,0
3500,"Donald Trump likes feuds. He cannot, for any r...",1


In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
ps = WordNetLemmatizer()
stopwords = stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def cleaning_data(row):
    
    # convert text to into lower case
    row = row.lower() 
    
    # this line of code only take words from text and remove number and special character using RegX
    row = re.sub('[^a-zA-Z]' , ' ' , row)
    
    # split the data and make token.
    token = row.split() 
    
    # lemmatize the word and remove stop words like a, an , the , is ,are ...
    news = [ps.lemmatize(word) for word in token if not word in stopwords]  
    
    # finaly join all the token with space
    cleanned_news = ' '.join(news) 
    
    # return cleanned data
    return cleanned_news

In [ ]:
dataset['text'] = dataset['text'].apply(lambda x : cleaning_data(x))

dataset.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [ ]:
dataset.shape

(44898, 2)

In [ ]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [ ]:
X.head()

14181    audio karl rove hugh hewitt picking fresh face...
20919    hysterical u marine enters catholic church con...
1655     alex jones conspiracy theory website infowars ...
14675    church organization church organization take m...
14862    green criminal working distribute money far aw...
Name: text, dtype: object

In [ ]:
y.head()

14181    1
20919    1
1655     1
14675    1
14862    1
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
train_data , test_data , train_label , test_label = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [ ]:
vec_train_data = vectorizer.fit_transform(train_data)

In [ ]:
vec_train_data = vec_train_data.toarray()

In [ ]:
train_data.shape , test_data.shape

((28000,), (7000,))

In [ ]:
vec_test_data = vectorizer.transform(test_data).toarray()

In [ ]:
vec_train_data.shape , vec_test_data.shape

((28000, 50000), (7000, 50000))

In [ ]:
train_label.value_counts() # balanced partition

1    14700
0    13300
Name: label, dtype: int64

In [ ]:
test_label.value_counts() # balanced partition

1    3685
0    3315
Name: label, dtype: int64

In [ ]:
training_data = pd.DataFrame(vec_train_data , columns=vectorizer.get_feature_names())
testing_data = pd.DataFrame(vec_test_data , columns= vectorizer.get_feature_names())


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
clf = MultinomialNB()
clf.fit(training_data, train_label)
y_pred  = clf.predict(testing_data)

In [ ]:
pd.Series(y_pred).value_counts()

1    3731
0    3269
dtype: int64

In [ ]:
test_label.value_counts()

1    3685
0    3315
Name: label, dtype: int64

In [ ]:
print(classification_report(test_label , y_pred))

              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3315
           1       0.95      0.96      0.95      3685

    accuracy                           0.95      7000
   macro avg       0.95      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



In [ ]:
y_pred_train = clf.predict(training_data)
print(classification_report(train_label , y_pred_train))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13300
           1       0.96      0.96      0.96     14700

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [ ]:
accuracy_score(train_label , y_pred_train)



0.9598928571428571

In [ ]:
accuracy_score(test_label , y_pred)

0.9522857142857143

In [ ]:
news = cleaning_data(str("Pelosi said that despite communist China’s long list of human rights abuses, the U.S. must partner with them to fight climate change."))

In [ ]:
single_prediction = clf.predict(vectorizer.transform([news]).toarray())
single_prediction

array([0])